# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
data = pd.DataFrame({'user_id': ['u1', 'u2', 'u3'], 
                     'recommended_list': [[143, 156, 1134, 991, 27], [1543, 3345, 533, 11, 43],
                                          [156, 3345, 10, 15, 1134]],
                     'prices_recommended': [[400, 60, 40, 40, 90] for i in range(3)],
                     'bought_list': [[156, 27], [11, 43], [1]],
                     'prices_bought': [[60, 90], [40, 90], [400]]})
data

,user_id,recommended_list,prices_recommended,bought_list,prices_bought
0,u1,"[143, 156, 1134, 991, 27]","[400, 60, 40, 40, 90]","[156, 27]","[60, 90]"
1,u2,"[1543, 3345, 533, 11, 43]","[400, 60, 40, 40, 90]","[11, 43]","[40, 90]"
2,u3,"[156, 3345, 10, 15, 1134]","[400, 60, 40, 40, 90]",[1],[400]


## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [3]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)

    flags = np.isin(bought_list, recommended_list)
    index_true = np.nonzero(flags)[0]
    
    recall = sum(prices_bought[index_true]) / sum(prices_recommended)
    
    return recall

In [4]:
data['recall@4'] = [recall_at_k(data.loc[i, 'recommended_list'], 
                                data.loc[i, 'bought_list'], 4) for i in range(3)]

data

,user_id,recommended_list,prices_recommended,bought_list,prices_bought,recall@4
0,u1,"[143, 156, 1134, 991, 27]","[400, 60, 40, 40, 90]","[156, 27]","[60, 90]",0.5
1,u2,"[1543, 3345, 533, 11, 43]","[400, 60, 40, 40, 90]","[11, 43]","[40, 90]",0.5
2,u3,"[156, 3345, 10, 15, 1134]","[400, 60, 40, 40, 90]",[1],[400],0.0


In [5]:
data['money_recall@4'] = [money_recall_at_k(data.loc[i, 'recommended_list'], 
                                            data.loc[i, 'bought_list'], 
                                            data.loc[i, 'prices_recommended'],
                                            data.loc[i, 'prices_bought'], 5) for i in range(3)]
data

,user_id,recommended_list,prices_recommended,bought_list,prices_bought,recall@4,money_recall@4
0,u1,"[143, 156, 1134, 991, 27]","[400, 60, 40, 40, 90]","[156, 27]","[60, 90]",0.5,0.238095
1,u2,"[1543, 3345, 533, 11, 43]","[400, 60, 40, 40, 90]","[11, 43]","[40, 90]",0.5,0.206349
2,u3,"[156, 3345, 10, 15, 1134]","[400, 60, 40, 40, 90]",[1],[400],0.0,0.000000


## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  ReciprocalRank(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [6]:
def mrr_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    index_true = np.nonzero(flags)[0]
    index_first = index_true[0] if len(index_true) else -1

    mrr = 1 / (index_first + 1) if index_first >= 0 else 0

    return mrr

In [7]:
data['mrr@4'] = [mrr_at_k(data.loc[i, 'recommended_list'], data.loc[i, 'bought_list'], 5) 
                 for i in range(3)]
data

,user_id,recommended_list,prices_recommended,bought_list,prices_bought,recall@4,money_recall@4,mrr@4
0,u1,"[143, 156, 1134, 991, 27]","[400, 60, 40, 40, 90]","[156, 27]","[60, 90]",0.5,0.238095,0.50
1,u2,"[1543, 3345, 533, 11, 43]","[400, 60, 40, 40, 90]","[11, 43]","[40, 90]",0.5,0.206349,0.25
2,u3,"[156, 3345, 10, 15, 1134]","[400, 60, 40, 40, 90]",[1],[400],0.0,0.000000,0.00


## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [8]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    index_true = np.nonzero(flags)[0]
    
    dcg, idcg = 0, 0
    for i in index_true:
        dcg += 1 / math.log2(i + 2)
    for k in range(len(bought_list)):
        idcg += 1 / math.log2(k + 2)
    ndcg = dcg / idcg
        
    return ndcg

In [9]:
data['ndcg@4'] = [ndcg_at_k(data.loc[i, 'recommended_list'], 
                            data.loc[i, 'bought_list'], 4) for i in range(3)]

data

,user_id,recommended_list,prices_recommended,bought_list,prices_bought,recall@4,money_recall@4,mrr@4,ndcg@4
0,u1,"[143, 156, 1134, 991, 27]","[400, 60, 40, 40, 90]","[156, 27]","[60, 90]",0.5,0.238095,0.50,0.624051
1,u2,"[1543, 3345, 533, 11, 43]","[400, 60, 40, 40, 90]","[11, 43]","[40, 90]",0.5,0.206349,0.25,0.501266
2,u3,"[156, 3345, 10, 15, 1134]","[400, 60, 40, 40, 90]",[1],[400],0.0,0.000000,0.00,0.000000
